In [7]:
!python3 -m pip install requests  --user
!python3 -m pip install beautifulsoup4 --user

"""黃金屋爬蟲"""
import asyncio
import re

import requests
from bs4 import BeautifulSoup

BASE_URL = "https://tw.hjwzw.com"
MOBILE_URL = "https://t.hjwzw.com"


You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Repo
[Telegram Bot](https://github.com/python-telegram-bot/python-telegram-bot)  
[OLAMI Medium](https://medium.com/@zaoldyeck/%E5%AF%A6%E6%88%B0%E7%AF%87-%E6%89%93%E9%80%A0%E4%BA%BA%E6%80%A7%E5%8C%96-telegram-bot-ed9bb5b8a6d9)

---

In [8]:
async def search_title(title:str) -> list:
    """取得搜尋資料"""
    url = f"{BASE_URL}/List/{title}"
    
    res = requests.get(url)
    if res.status_code != requests.codes.ok:
        return "Error" # 網站不正常

    soup = BeautifulSoup(res.text, "html.parser")

    # 是直接搜到主頁嗎
    main = soup.find_all("a", string=re.compile(f"{title}"))
    is_main =  main != None and all([main[0]["href"] in i["href"] for i in main])

    if is_main:
        main = main[0]
        list_ = await get_book_info(main.text.strip(), main["href"])
    else:
        books = soup.find_all("span", class_="wd10")
        list_ = await asyncio.gather(
            *[get_book_info(i.a.text, i.a["href"]) for i in books]
        )
    return list_

In [9]:
async def get_book_info(title:str, url:str) -> dict:
    """取得書本資料"""
    url = f"{MOBILE_URL}{url}"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    return {
        "title": title,
        "url": url,
        "author": soup.find("span", string="作者：").findNext("span").text.strip(),
        "preface": soup.find("div", id="Contents").text.strip(),
        "id": url.split("/")[-1]
    }

async def get_chapter_list(book_id:str, start:int=0, end:int=100) -> list:
    """取得章節資料"""
    url = f"{MOBILE_URL}/ChapterList/{book_id}/{start}_{end}"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    chapters = soup.find_all("div", class_="book_list_03")

    if len(chapters) != (end-start):
        return 
    
    list_ = []
    for chapter in chapters:
        list_.append({
            "title": chapter.text.strip()
        })

    return list_
    
list_ = await search_title("九星")
print(list_[0])
await get_chapter_list(list_[0]["id"])


{'title': '九星霸體訣', 'url': 'https://t.hjwzw.com/Book/36508', 'author': '平凡魔術師', 'preface': '是丹帝重生？是融合靈魂？被盜走靈根、靈血、靈骨的三無少年——龍塵，憑借著記憶中的煉丹神術，修行神秘功法九星霸體訣，撥開重重迷霧，解開驚天之局。\r\n手掌天地乾坤，腳踏日月星辰，勾搭各色美女，鎮壓惡鬼邪神。\r\n江湖傳聞：龍塵一到，地吼天嘯。龍塵一出，鬼泣神哭。\r\n本故事純屬虛構，如有雷同，那就是真事兒.', 'id': '36508'}


[{'title': '第一章 丹帝記憶'},
 {'title': '第二章 卑鄙小人'},
 {'title': '第三章 鋒芒初現'},
 {'title': '第四章 凝聚風府星'},
 {'title': '第五章 太學宮'},
 {'title': '第六章 討還利息'},
 {'title': '第七章 修習戰技'},
 {'title': '第八章 生死決戰'},
 {'title': '第九章 煉藥師公會'},
 {'title': '第十章 懲戒小人'},
 {'title': '第十一章 絕世美人'},
 {'title': '第十二章 新的目標'},
 {'title': '第十三章 風府初成'},
 {'title': '第十四章 七皇子'},
 {'title': '第十五章 開天'},
 {'title': '第十六章 公主尋仇'},
 {'title': '第十七章 殺意凜然'},
 {'title': '第十八章 蠻荒候'},
 {'title': '第十九章 駐顏液'},
 {'title': '第二十章 賒賬'},
 {'title': '第二十一章 考核'},
 {'title': '第二十二章 阿蠻'},
 {'title': '第二十三章 膽上長毛'},
 {'title': '第二十四章 報仇不隔夜'},
 {'title': '第二十五章 大夏皇子'},
 {'title': '第二十六章 恐怖阿蠻'},
 {'title': '第二十七章 火云掌'},
 {'title': '第二十八章 公主秘辛'},
 {'title': '第二十九章 龍塵的震驚'},
 {'title': '第三十章 死性不改'},
 {'title': '第三十一章 再起沖突'},
 {'title': '第三十二章 華云閣的邀請'},
 {'title': '第三十三章 鳳鳴燈節'},
 {'title': '第三十四章 燈節開幕'},
 {'title': '第三十五章 硝煙彌漫'},
 {'title': '第三十六章 公主芳心'},
 {'title': '第三十七章 斗丹'},
 {'title': '第三十八章 輸了?'},
 {'title': '第三十九章 獸火到手'},
 {'title': '第四十章 黃裳的挑戰'},
 